# Chords Progressions Transformer Aux (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2024

***

# (GPU CHECK)

In [ ]:
#@title NVIDIA GPU check
!nvidia-smi

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install dependencies
!git clone --depth 1 https://github.com/asigalov61/Chords-Progressions-Transformer
!pip install huggingface_hub
!pip install einops
!pip install torch-summary
!apt install fluidsynth #Pip does not work for some reason. Only apt works

In [ ]:
#@title Import modules

print('=' * 70)
print('Loading core Chords Progressions Transformer modules...')

import os
import copy
import pickle
import secrets
import statistics
from time import time
import tqdm

print('=' * 70)
print('Loading main Chords Progressions Transformer modules...')
import torch

%cd /content/Chords-Progressions-Transformer

import TMIDIX

from midi_to_colab_audio import midi_to_colab_audio

from x_transformer_1_23_2 import *

import random

%cd /content/
print('=' * 70)
print('Loading aux Chords Progressions Transformer modules...')

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

from IPython.display import Audio, display

from huggingface_hub import hf_hub_download

from google.colab import files

print('=' * 70)
print('Done!')
print('Enjoy! :)')
print('=' * 70)

# (LOAD MODEL)

In [ ]:
#@title Load Chords Progressions Transformer Pre-Trained Model

#@markdown Choose model

select_model_to_load = "187M-2048E-4L-16H-FP32-Fast-Small" # @param ["93M-1024E-8L-8H-Very-Fast-Small", "187M-2048E-4L-16H-Fast-Small", "187M-2048E-4L-16H-FP32-Fast-Small"]

#@markdown Model precision option

model_precision = "bfloat16" # @param ["bfloat16", "float16", "float32"]

#@markdown bfloat16 == Half precision/faster speed (if supported, otherwise the model will default to float16)

#@markdown float16 == Full precision/fast speed

plot_tokens_embeddings = False # @param {type:"boolean"}

print('=' * 70)
print('Loading Chords Progressions Transformer Pre-Trained Model...')
print('Please wait...')
print('=' * 70)

full_path_to_models_dir = "/content/Chords-Progressions-Transformer/Models"

if select_model_to_load == '93M-1024E-8L-8H-Very-Fast-Small':

  dim = 1024
  depth = 8
  heads = 8

  model_checkpoint_file_name = 'Chords_Progressions_Transformer_Small_Trained_Model_9609_steps_1.0704_loss_0.6927_acc.pth'
  model_path = full_path_to_models_dir+'/Small/'+model_checkpoint_file_name
  if os.path.isfile(model_path):
    print('Model already exists...')

  else:
    hf_hub_download(repo_id='asigalov61/Chords-Progressions-Transformer',
                    filename=model_checkpoint_file_name,
                    local_dir='/content/Chords-Progressions-Transformer/Models/Small',
                    local_dir_use_symlinks=False)

elif select_model_to_load == '187M-2048E-4L-16H-Fast-Small':

  dim = 2048
  depth = 4
  heads = 16

  model_checkpoint_file_name = 'Chords_Progressions_Transformer_Small_2048_Trained_Model_12947_steps_0.9316_loss_0.7386_acc.pth'
  model_path = full_path_to_models_dir+'/Small_2048/'+model_checkpoint_file_name
  if os.path.isfile(model_path):
    print('Model already exists...')

  else:
    hf_hub_download(repo_id='asigalov61/Chords-Progressions-Transformer',
                    filename=model_checkpoint_file_name,
                    local_dir='/content/Chords-Progressions-Transformer/Models/Small_2048',
                    local_dir_use_symlinks=False)

elif select_model_to_load == '187M-2048E-4L-16H-FP32-Fast-Small':

  dim = 2048
  depth = 4
  heads = 16

  model_checkpoint_file_name = 'Chords_Progressions_Transformer_Small_2048_FP32_Trained_Model_6265_steps_0.9272_loss_0.7369_acc.pth'
  model_path = full_path_to_models_dir+'/Small_2048_FP32/'+model_checkpoint_file_name
  if os.path.isfile(model_path):
    print('Model already exists...')

  else:
    hf_hub_download(repo_id='asigalov61/Chords-Progressions-Transformer',
                    filename=model_checkpoint_file_name,
                    local_dir='/content/Chords-Progressions-Transformer/Models/Small_2048_FP32',
                    local_dir_use_symlinks=False)

print('=' * 70)
print('Instantiating model...')

device_type = 'cuda'

if model_precision == 'bfloat16' and torch.cuda.is_bf16_supported():
  dtype = 'bfloat16'
else:
  dtype = 'float16'

if model_precision == 'float16':
  dtype = 'float16'

if model_precision == 'float32':
  dtype = 'float32'

ptdtype = {'bfloat16': torch.bfloat16, 'float16': torch.float16, 'float32': torch.float32}[dtype]
ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype)

SEQ_LEN = 8192 # Models seq len
PAD_IDX = 707 # Models pad index

# instantiate the model

model = TransformerWrapper(
    num_tokens = PAD_IDX+1,
    max_seq_len = SEQ_LEN,
    attn_layers = Decoder(dim = dim, depth = depth, heads = heads, attn_flash = True)
    )

model = AutoregressiveWrapper(model, ignore_index = PAD_IDX)

model.cuda()
print('=' * 70)

print('Loading model checkpoint...')

model.load_state_dict(torch.load(model_path))
print('=' * 70)

model.eval()

print('Done!')
print('=' * 70)

print('Model will use', dtype, 'precision...')
print('=' * 70)

# Model stats
print('Model summary...')
summary(model)

# Plot Token Embeddings
if plot_tokens_embeddings:
  tok_emb = model.net.token_emb.emb.weight.detach().cpu().tolist()

  cos_sim = metrics.pairwise_distances(
    tok_emb, metric='cosine'
  )
  plt.figure(figsize=(7, 7))
  plt.imshow(cos_sim, cmap="inferno", interpolation="nearest")
  im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
  plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
  plt.xlabel("Position")
  plt.ylabel("Position")
  plt.tight_layout()
  plt.plot()
  plt.savefig("/content/Chords-Progressions-Transformer-Tokens-Embeddings-Plot.png", bbox_inches="tight")

print('=' * 70)
print('Loading Chords Progressions Transformer Aux Model...')
print('=' * 70)

aux_model_path = '/content/Chords-Progressions-Transformer/Models/Aux/Chords_Progressions_Transformer_Aux_Trained_Model_4374_steps_0.1185_loss_0.9673_acc.pth'

if os.path.isfile(aux_model_path):
  print('Model already exists...')

else:
  hf_hub_download(repo_id='asigalov61/Chords-Progressions-Transformer',
                  filename='Chords_Progressions_Transformer_Aux_Trained_Model_4374_steps_0.1185_loss_0.9673_acc.pth',
                  local_dir='/content/Chords-Progressions-Transformer/Models/Aux',
                  local_dir_use_symlinks=False)

AUX_SEQ_LEN = 1024 # Models seq len
AUX_PAD_IDX = 322 # Models pad index

# instantiate the model

aux_model = TransformerWrapper(
            num_tokens = AUX_PAD_IDX+1,
            max_seq_len = AUX_SEQ_LEN,
            attn_layers = Decoder(dim = 1024, depth = 4, heads = 16, attn_flash = True)
            )

aux_model = AutoregressiveWrapper(aux_model, ignore_index = AUX_PAD_IDX)

aux_model.cuda()
print('=' * 70)

print('Loading model checkpoint...')

aux_model.load_state_dict(torch.load(aux_model_path))
print('=' * 70)

aux_model.eval()

# Model stats
print('Model summary...')
summary(aux_model)

print('Done!')
print('=' * 70)

# (CHORDS PROGRESSIONS)

In [ ]:
# @title Generate chords progressions from custom MIDI chords

#@markdown NOTE: You can stop the generation at any time to render partial results

#@markdown Generation settings

output_MIDI_patch_number = 0 # @param {type:"slider", min:0, max:127, step:1}
number_of_chords_to_generate = 128 # @param {type:"slider", min:8, max:1020, step:1}
max_number_of_notes_per_chord = 8 # @param {type:"slider", min:1, max:10, step:1}
number_of_memory_tokens = 4096 # @param {type:"slider", min:32, max:8188, step:4}
temperature = 0.9 # @param {type:"slider", min:0.1, max:1, step:0.05}

#@markdown Other settings

render_MIDI_to_audio = True # @param {type:"boolean"}

#===============================================================================

def generate_chords(chords,
                    max_chords_limit = 8,
                    num_memory_tokens = 4096,
                    temperature=0.9
                    ):

    chords = chords[-num_memory_tokens:]

    x = torch.tensor([chords] * 1, dtype=torch.long, device='cuda')

    o = 0

    ncount = 0

    while o < 384 and ncount < max_chords_limit:
      with ctx:
        out = model.generate(x,
                            1,
                            temperature=temperature,
                            return_prime=False,
                            verbose=False)

      o = out.tolist()[0][0]

      if 256 <= o < 384:
        ncount += 1

      if o < 384:
        x = torch.cat((x, out), 1)

    return x.tolist()[0][len(chords):]

#===============================================================================

print('=' * 70)
print('Chords Progressions Transformer Aux Model Generator')
print('=' * 70)

torch.cuda.empty_cache()

x = torch.tensor([[321]] * 1, dtype=torch.long, device='cuda')

out = aux_model.generate(x,
                        number_of_chords_to_generate,
                        temperature=temperature,
                        return_prime=False,
                        verbose=True)

chords = [c+384 for c in out.tolist()[0]]
#===============================================================================

print('=' * 70)
print('Chords Progressions Transformer Primary Model Generator')
print('=' * 70)

torch.cuda.empty_cache()

output = []

idx = 0

for c in tqdm.tqdm(chords):

  try:

    output.append(c)

    out = generate_chords(output,
                        temperature=temperature,
                        max_chords_limit=max_number_of_notes_per_chord,
                        num_memory_tokens=number_of_memory_tokens
                        )
    output.extend(out)

    idx += 1

  except KeyboardInterrupt:
    print('=' * 70)
    print('Stopping generation...')
    break

  except Exception as e:
    print('=' * 70)
    print('Error:', e)
    break

torch.cuda.empty_cache()

#===============================================================================
print('=' * 70)

out1 = output

print('Sample INTs', out1[:12])
print('=' * 70)

if len(out) != 0:

    song = out1
    song_f = []

    time = 0
    dur = 0
    vel = 90
    pitch = 0
    channel = 0

    patches = [0] * 16
    patches[0] = output_MIDI_patch_number

    for ss in song:

        if 0 <= ss < 128:

            time += ss * 32

        if 128 <= ss < 256:

            dur = (ss-128) * 32

        if 256 <= ss < 384:

            pitch = (ss-256)

            vel = max(40, pitch)

            song_f.append(['note', time, dur, channel, pitch, vel, output_MIDI_patch_number])

    detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                            output_signature = 'Chords Progressions Transformer',
                                                            output_file_name = '/content/Chords-Progressions-Transformer-Composition',
                                                            track_name='Project Los Angeles',
                                                            list_of_MIDI_patches=patches
                                                            )



    print('=' * 70)
    print('Displaying resulting composition...')
    print('=' * 70)

    fname = '/content/Chords-Progressions-Transformer-Composition'

    if render_MIDI_to_audio:
      midi_audio = midi_to_colab_audio(fname + '.mid')
      display(Audio(midi_audio, rate=16000, normalize=False))

    TMIDIX.plot_ms_SONG(song_f, plot_title=fname)

# Congrats! You did it! :)